# Hello, World
This is the dog breed classifying machine learning model.

In [23]:
!pip install kaggle
import zipfile
import pandas as pd
import numpy as np
import scipy
# import kagglehub
# import os
# from dotenv import load_dotenv


# load_dotenv()
# print(os.environ.get('KAGGLE_USERNAME'))

# path = kagglehub.dataset_download("jessicali9530/stanford-dogs-dataset")

In [25]:
# Import data
!kaggle datasets download -d jessicali9530/stanford-dogs-dataset

Dataset URL: https://www.kaggle.com/datasets/jessicali9530/stanford-dogs-dataset
License(s): other




  0%|          | 0.00/750M [00:00<?, ?B/s]
  0%|          | 1.00M/750M [00:02<30:04, 436kB/s]
  0%|          | 2.00M/750M [00:03<23:31, 556kB/s]
  0%|          | 3.00M/750M [00:05<22:14, 587kB/s]
  1%|          | 4.00M/750M [00:07<21:13, 615kB/s]
  1%|          | 5.00M/750M [00:09<21:48, 598kB/s]
  1%|          | 6.00M/750M [00:10<19:18, 674kB/s]
  1%|          | 7.00M/750M [00:12<21:27, 606kB/s]
  1%|          | 8.00M/750M [00:15<27:45, 467kB/s]
  1%|          | 9.00M/750M [00:17<25:28, 509kB/s]
  1%|▏         | 10.0M/750M [00:17<19:02, 680kB/s]
  1%|▏         | 11.0M/750M [00:17<14:20, 901kB/s]
  2%|▏         | 12.0M/750M [00:18<10:34, 1.22MB/s]
  2%|▏         | 13.0M/750M [00:18<08:01, 1.61MB/s]
  2%|▏         | 14.0M/750M [00:18<06:16, 2.05MB/s]
  2%|▏         | 15.0M/750M [00:18<05:02, 2.55MB/s]
  2%|▏         | 16.0M/750M [00:18<04:10, 3.07MB/s]
  2%|▏         | 17.0M/750M [00:19<03:30, 3.65MB/s]
  2%|▏         | 18.0M/750M [00:19<03:03, 4.18MB/s]
  3%|▎         | 19.0M/750M [00

In [26]:
# Unzip file for use
with zipfile.ZipFile("stanford-dogs-dataset.zip", "r") as z:
  z.extractall("sd_dataset")